**Helper functions and installing libraries**

In [100]:
import requests
import pandas as pd
import time
from pygame import mixer #for sound
from datetime import datetime, timedelta #for looping over date

In [113]:
API_key = 'JTFthLtGYuKFQK47MPvQcrabXJRJPk5nUEgaric4f17oP5VJcHyHqsqCMWXu'

#append to dictipnary 
def app_dictionary(id, list, dictionary):
    if id not in dictionary: #checks if column with key already exists 
        dictionary[id] = [list]
    else:
        dictionary[id].append(list) #if yes, it appends to key value

#running api calls and returning result + pagination
def run_api(api_call):
    try:
        result = requests.get(api_call)
        result = result.json()
    except ValueError:
        print(f"Failed to decode JSON. Please check API call: {api_call}")

    #checks if there is a next page
    #if yes it returns true, if no it returns false
    try:
        has_more = result['pagination']['has_more']
    except:
        has_more = False

    try:
        rate_limit = result['rate_limit']['remaining']
        reset = result['rate_limit']['resets_in_seconds']
    except:
        print('no rate limit available')

    return result, has_more, rate_limit, reset

#function for playing sound
def play_sound():
    mixer.init()
    mixer.music.load('done-for-you.mp3')
    mixer.music.play()

def run_pages(api_first, api_second):
    results, has_more, rate_limit, reset = run_api(f"{api_first}{api_second}")

    page = 2
    while has_more == True and int(rate_limit) > 0:
        next_results, has_more, rate_limit, reset = run_api(f"{api_first}page={page}&{api_second}")
        results['data'] = results['data'] + next_results['data']
        page += 1
    else:
        time.sleep(int(reset))
        
    return results

**Get league names**

In [8]:
leagues = requests.get(f"https://api.sportmonks.com/v3/football/leagues?api_token={API_key}")

leagues = leagues.json()

leagues

{'data': [{'id': 8,
   'sport_id': 1,
   'country_id': 462,
   'name': 'Premier League',
   'active': True,
   'short_code': 'UK PL',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/leagues/8/8.png',
   'type': 'league',
   'sub_type': 'domestic',
   'last_played_at': '2024-10-27 14:00:00',
   'category': 1,
   'has_jerseys': False},
  {'id': 9,
   'sport_id': 1,
   'country_id': 462,
   'name': 'Championship',
   'active': True,
   'short_code': 'UK Champ',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/leagues/9/9.png',
   'type': 'league',
   'sub_type': 'domestic',
   'last_played_at': '2024-10-27 15:00:00',
   'category': 2,
   'has_jerseys': False},
  {'id': 24,
   'sport_id': 1,
   'country_id': 462,
   'name': 'FA Cup',
   'active': True,
   'short_code': 'UK FA Cup',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/leagues/24/24.png',
   'type': 'league',
   'sub_type': 'domestic_cup',
   'last_played_at': '2024-10-15 18:45:00',
   'category': 

In [12]:
wanted_leagues = ['Premier League', 'La Liga', 'Bundesliga', 'Serie A']
wanted_features = ['id', 'country_id', 'name', 'image_path']

leagues_df = {}

for league in leagues['data']:
    if league['name'] in wanted_leagues:
        for feature in wanted_features:
            app_dictionary(feature, league[f'{feature}'], leagues_df)
        
leagues_df = pd.DataFrame(leagues_df)
leagues_df.to_csv('data/leagues.csv')

**Get season ID by search**

In [16]:
season_names = ['2023', '2021', '2019', '2017', '2015', '2013']
season_data = {'data':[]}
leagues_df = pd.read_csv('data/leagues.csv')  
league_id = leagues_df['id'].to_numpy()

for season_name in season_names:
    seasons, has_more = run_api(f"https://api.sportmonks.com/v3/football/seasons/search/{season_name}?api_token={API_key}")
    
    page = 2
    while has_more == True:
        next_seasons, has_more = run_api(f"https://api.sportmonks.com/v3/football/seasons/search/{season_name}?page={page}&api_token={API_key}")
        seasons['data'] = seasons['data'] + next_seasons['data']
        page += 1
    
    for season in seasons['data']:
        if season['league_id'] in league_id:
            season_data['data'].append(season)

for season in season_data['data']:
    if season['league_id'] in league_id:
        print(season)

{'id': 19734, 'sport_id': 1, 'league_id': 8, 'tie_breaker_rule_id': 1526, 'name': '2022/2023', 'finished': True, 'pending': False, 'is_current': False, 'starting_at': '2022-08-05', 'ending_at': '2023-05-28', 'standings_recalculated_at': '2023-11-30 15:26:55', 'games_in_current_week': False}
{'id': 19744, 'sport_id': 1, 'league_id': 82, 'tie_breaker_rule_id': 171, 'name': '2022/2023', 'finished': True, 'pending': False, 'is_current': False, 'starting_at': '2022-08-05', 'ending_at': '2023-05-27', 'standings_recalculated_at': '2023-11-30 15:44:34', 'games_in_current_week': False}
{'id': 19781, 'sport_id': 1, 'league_id': 486, 'tie_breaker_rule_id': 171, 'name': '2022/2023', 'finished': True, 'pending': False, 'is_current': False, 'starting_at': '2022-07-15', 'ending_at': '2023-06-03', 'standings_recalculated_at': '2023-06-03 16:01:08', 'games_in_current_week': False}
{'id': 19799, 'sport_id': 1, 'league_id': 564, 'tie_breaker_rule_id': 170, 'name': '2022/2023', 'finished': True, 'pending'

In [18]:
seasonByLeagueID = pd.DataFrame(season_data['data'])
seasonByLeagueID.to_csv('data/seasons.csv')

Get venues by Season ID

In [31]:
venues_data = {'data':[]}
seasons_df = pd.read_csv('data/seasons.csv')  
seasons_ids = seasons_df['id'].to_numpy()

for season_id in seasons_ids:
    first = 'https://api.sportmonks.com/v3/football/venues/seasons/' + str(season_id) + '?'
    second = 'api_token=' + API_key
    results = run_pages(first, second)
    
    try:
        for data in results['data']:
            venues_data['data'].append(data)
    except KeyError as e:
        print(f'KeyError at id: {season_id}')

In [35]:
venues_data = pd.DataFrame(venues_data['data'])
venues_data.to_csv('data/venues.csv')

Get Teams by Season ID

In [41]:
season_id = seasonByLeagueID['id'].to_numpy()
teams_data = {'data':[]}

for id in season_id:
    first = 'https://api.sportmonks.com/v3/football/teams/seasons/' + str(id) + '?'
    second = 'api_token=' + API_key
    print(f'Season id: {id}')
    results = run_pages(first, second)
    
    for team in results['data']:
        team['season_id'] = id
        teams_data['data'].append(team)

Season id: 19734
Season id: 19744
Season id: 19781
Season id: 19799
Season id: 19806
Season id: 21646
Season id: 21694
Season id: 21713
Season id: 21795
Season id: 21818
Season id: 17288
Season id: 17361
Season id: 17420
Season id: 17480
Season id: 17488
Season id: 18375
Season id: 18378
Season id: 18444
Season id: 18462
Season id: 18576
Season id: 12962
Season id: 12996
Season id: 13005
Season id: 13133
Season id: 13158
Season id: 16021
Season id: 16036
Season id: 16264
Season id: 16326
Season id: 16415
Season id: 13
Season id: 219
Season id: 802
Season id: 828
Season id: 853
Season id: 6375
Season id: 6397
Season id: 8026
Season id: 8442
Season id: 8557
Season id: 10
Season id: 12
Season id: 217
Season id: 218
Season id: 1583
Season id: 1584
Season id: 1916
Season id: 1917
Season id: 2061
Season id: 2063
Season id: 3
Season id: 7
Season id: 215
Season id: 216
Season id: 1581
Season id: 1582
Season id: 1914
Season id: 1915
Season id: 2059
Season id: 2060


In [44]:
teams_df = pd.DataFrame(teams_data['data'])

In [43]:
teams_df.to_csv('data/teams.csv')

**Get fixtures by team**

In [78]:
seasons_df = pd.read_csv('data/seasons.csv')  
start_date = seasons_df.sort_values(by=['starting_at'])['starting_at'].to_numpy()[0]
end_date = seasons_df.sort_values(by=['ending_at'], ascending=False)['ending_at'].to_numpy()[0]
#2012-07-20
#2024-06-02

Looping over periods of 100 days (limitation of the API)

In [83]:
# Initial date (starting point)
start_date = datetime(2012, 7, 20)  # Starting date in 'year-month-day'

# Number of intervals and days
days_interval = 100
total_days = 4335

# Variable to store the previous date
previous_date = None

In [108]:
teams_id = teams_df['id'].unique()
fixture_data = {'data': []}

# Loop over the 300 days in 100-day intervals
for i in range(0, total_days, days_interval):
    current_date = start_date + timedelta(days=i)
    formatted_date = current_date.strftime('%Y-%m-%d')

    # Print the previous and current date
    if previous_date:
        print(f"Previous date: {previous_date} -> Next date: {formatted_date}")
        for id in teams_id:
            first = 'https://api.sportmonks.com/v3/football/fixtures/between/' + str(previous_date) + '/' + str(formatted_date) + '/' + str(id) + '?'
            second = '?api_token=' + API_key + '&per_page=50&include=formations;weatherReport;referees;events:type_id,player_name,addition,minute&filters=eventTypes:10,19,20,21;fixtureLeagues:8,82,384,486,564'
            results = run_pages(first, second)
            
            try:
                for fixture in results['data']:
                    fixture_data['data'].append(fixture)
            except:
                print(f'No data for team {id}')
    else: # Avoiding calling only the first date 
        print(f"First date: {formatted_date}") 

    # Update the previous date for the next iteration
    previous_date = formatted_date


Previous date: 2012-07-20 -> Next date: 2012-07-20
no rate limit available


UnboundLocalError: cannot access local variable 'rate_limit' where it is not associated with a value

In [ ]:
fixture_data = {'data': []}
start_date = '2012-09-20'
end_date = '2012-12-20'

for id in teams_id:
    first = 'https://api.sportmonks.com/v3/football/fixtures/between/' + start_date + '/' + end_date + '/' + str(id) + '?'
    second = 'api_token=' + API_key + '&per_page=50&include=formations;weatherReport;referees;events:type_id,player_name,addition,minute&filters=eventTypes:10,19,20,21;fixtureLeagues:8,82,384,486,564'
    results = run_pages(first, second)
    
    try:
        for fixture in results['data']:
            fixture_data['data'].append(fixture)
    except:
        print(f'No data for team {id}')

No data for team 78


In [97]:
play_sound()

Saving as csv

In [98]:
fixture_data = pd.DataFrame(fixture_data['data'])

fixture_data.to_csv('data/fixture_data_full_test.csv')

In [111]:
first = 'https://api.sportmonks.com/v3/football/fixtures/between/' + start_date + '/' + end_date + '/' + str(id) + '?'
second = 'api_token=' + API_key + '&per_page=50&include=formations;weatherReport;referees;events:type_id,player_name,addition,minute&filters=eventTypes:10,19,20,21;fixtureLeagues:8,82,384,486,564'
result, has_more, rate_limit, reset = run_api(f'https://api.sportmonks.com/v3/football/fixtures/between/{start_date}/{end_date}/{id}?')

TypeError: run_api() takes 1 positional argument but 2 were given

**API call for events and add ons**

In [184]:
next_features, has_more = run_api(f"https://api.sportmonks.com/v3/football/fixtures/18535049?api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep&per_page=50&include=formations;weatherReport;referees;events:type_id,player_name,addition,minute&filters=eventTypes:10,19,20,21")

next_features

No noxt page for api call https://api.sportmonks.com/v3/football/fixtures/18535049?api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep&per_page=50&include=formations;weatherReport;referees;events:type_id,player_name,addition,minute&filters=eventTypes:10,19,20,21


{'data': {'id': 18535049,
  'sport_id': 1,
  'league_id': 8,
  'season_id': 19734,
  'stage_id': 77457864,
  'group_id': None,
  'aggregate_id': None,
  'round_id': 274668,
  'state_id': 5,
  'venue_id': 206,
  'name': 'Manchester United vs Brighton & Hove Albion',
  'starting_at': '2022-08-07 13:00:00',
  'result_info': 'Brighton & Hove Albion won after full-time.',
  'leg': '1/1',
  'details': None,
  'length': 90,
  'placeholder': False,
  'has_odds': True,
  'has_premium_odds': False,
  'starting_at_timestamp': 1659877200,
  'formations': [{'id': 336844,
    'fixture_id': 18535049,
    'participant_id': 78,
    'formation': '3-4-2-1',
    'location': 'away'},
   {'id': 336843,
    'fixture_id': 18535049,
    'participant_id': 14,
    'formation': '4-2-3-1',
    'location': 'home'}],
  'weatherreport': {'id': 103325,
   'fixture_id': 18535049,
   'venue_id': 206,
   'temperature': {'day': 20.95,
    'morning': 14.48,
    'evening': 18.84,
    'night': 14.99},
   'feels_like': {'day'

In [108]:
last_request_time = time.time()
#function for optimizing speed performance 
def dynamic_sleep():
    # Calculate the time elapsed since the last request
    elapsed_time = time.time() - last_request_time
    
    # If less than 3 seconds have passed, sleep for the remaining time
    if elapsed_time < 3:
        time.sleep(3 - elapsed_time)

In [ ]:
weather_report = {}
referee_pr_fixture = {}
formation_pr_fixture = {}

#teams_data = {'data':[]}
for fixture in events['data']:
        fixture['weatherreport'].append(weather_report)
        fixture['referees'][3].append(referee_pr_fixture)
        fixture['formations'][:].append(formation_pr_fixture)

In [101]:
fixture_data = pd.read_csv('data/fixture_data_full_test.csv')  

In [84]:
extented_api_search = {'data':[]}

for id in fixture_data['id']:
    fixtures, has_more = run_api(f"https://api.sportmonks.com/v3/football/fixtures/{id}?api_token={API_key}&per_page=50&include=formations;weatherReport;referees;events:type_id,player_name,addition,minute&filters=eventTypes:10,19,20,21")
    
    page = 2
    while has_more == True:
        next_features, has_more = run_api(f"https://api.sportmonks.com/v3/football/fixtures/{id}?page={page}&api_token={API_key}&per_page=50&include=formations;weatherReport;referees;events:type_id,player_name,addition,minute&filters=eventTypes:10,19,20,21")
        fixtures['data'] = fixtures['data'] + next_features['data']
        page += 1
    
    try:
        extented_api_search['data'].append(fixtures['data'])
    except:
        print(f'Could not append for fixture: {id}')
    
    time.sleep(3)

{'message': "No result(s) found matching your request. Either the query did not return any results or you don't have access to it via your current subscription.", 'subscription': [{'meta': {'trial_ends_at': '2024-11-04 11:48:51', 'ends_at': '2024-11-21 10:53:06', 'current_timestamp': 1730069364}, 'plans': [{'plan': 'European Plan', 'sport': 'Football', 'category': 'Advanced'}], 'add_ons': [], 'widgets': []}], 'rate_limit': {'resets_in_seconds': 3600, 'remaining': 2999, 'requested_entity': 'Fixture'}, 'timezone': 'UTC'}
{'message': "No result(s) found matching your request. Either the query did not return any results or you don't have access to it via your current subscription.", 'subscription': [{'meta': {'trial_ends_at': '2024-11-04 11:48:51', 'ends_at': '2024-11-21 10:53:06', 'current_timestamp': 1730069364}, 'plans': [{'plan': 'European Plan', 'sport': 'Football', 'category': 'Advanced'}], 'add_ons': [], 'widgets': []}], 'rate_limit': {'resets_in_seconds': 3600, 'remaining': 2998, '

In [107]:
extented_api_search = {'data':[]}

fixtures, has_more = run_api(f"https://api.sportmonks.com/v3/football/fixtures/18546372?api_token={API_key}&per_page=50&include=formations;weatherReport;referees;events:type_id,player_name,addition,minute&filters=eventTypes:10,19,20,21")

fixtures

{'data': {'id': 18546372,
  'sport_id': 1,
  'league_id': 384,
  'season_id': 19806,
  'stage_id': 77458057,
  'group_id': None,
  'aggregate_id': None,
  'round_id': 276020,
  'state_id': 5,
  'venue_id': 7305,
  'name': 'Spezia vs Juventus',
  'starting_at': '2023-02-19 17:00:00',
  'result_info': 'Juventus won after full-time.',
  'leg': '1/1',
  'details': None,
  'length': 90,
  'placeholder': False,
  'has_odds': True,
  'has_premium_odds': False,
  'starting_at_timestamp': 1676826000,
  'formations': [{'id': 395126,
    'fixture_id': 18546372,
    'participant_id': 625,
    'formation': '3-5-2',
    'location': 'away'},
   {'id': 395127,
    'fixture_id': 18546372,
    'participant_id': 345,
    'formation': '3-5-2',
    'location': 'home'}],
  'weatherreport': {'id': 168895,
   'fixture_id': 18546372,
   'venue_id': 7305,
   'temperature': {'day': 11.77,
    'morning': 10.58,
    'evening': 11.5,
    'night': 11.13},
   'feels_like': {'day': 11.51,
    'morning': 10.28,
    'ev

In [105]:
try:
    extented_api_search['data'].append(fixtures['data'])
except:
    print(f'Could not append for fixture: {id}')

In [106]:
extented_api_search

{'data': [{'id': 18546355,
   'sport_id': 1,
   'league_id': 384,
   'season_id': 19806,
   'stage_id': 77458057,
   'group_id': None,
   'aggregate_id': None,
   'round_id': 276019,
   'state_id': 5,
   'venue_id': 7220,
   'name': 'Empoli vs Spezia',
   'starting_at': '2023-02-11 14:00:00',
   'result_info': 'Game ended in draw.',
   'leg': '1/1',
   'details': None,
   'length': 90,
   'placeholder': False,
   'has_odds': True,
   'has_premium_odds': False,
   'starting_at_timestamp': 1676124000,
   'formations': [{'id': 392574,
     'fixture_id': 18546355,
     'participant_id': 345,
     'formation': '3-5-2',
     'location': 'away'},
    {'id': 392575,
     'fixture_id': 18546355,
     'participant_id': 397,
     'formation': '4-3-1-2',
     'location': 'home'}],
   'weatherreport': {'id': 164121,
    'fixture_id': 18546355,
    'venue_id': 7220,
    'temperature': {'day': 8.46,
     'morning': 0.86,
     'evening': 6.67,
     'night': 3.65},
    'feels_like': {'day': 8.46,
     

In [70]:
result = result.json()

In [71]:
result

{'message': "No result(s) found matching your request. Either the query did not return any results or you don't have access to it via your current subscription.",
 'subscription': [{'meta': {'trial_ends_at': '2024-11-04 11:48:51',
    'ends_at': '2024-11-21 10:53:06',
    'current_timestamp': 1729590039},
   'plans': [{'plan': 'European Plan',
     'sport': 'Football',
     'category': 'Advanced'}],
   'add_ons': [],
   'widgets': []}],
 'rate_limit': {'resets_in_seconds': 1106,
  'remaining': 47,
  'requested_entity': 'Fixture'},
 'timezone': 'UTC'}

In [51]:
play_sound()

In [52]:
weather_report = []
referee_pr_fixture = []
formation_pr_fixture = []

#teams_data = {'data':[]}
for fixture in extented_api_search['data']:
        weather_report.append(fixture['weatherreport'])
        formation_pr_fixture.append( fixture['formations'][:])

In [58]:
len(fixture_data['id'])

6171

In [36]:
events

{'data': [{'id': 18535054,
   'sport_id': 1,
   'league_id': 8,
   'season_id': 19734,
   'stage_id': 77457864,
   'group_id': None,
   'aggregate_id': None,
   'round_id': 274669,
   'state_id': 5,
   'venue_id': 480,
   'name': 'Brighton & Hove Albion vs Newcastle United',
   'starting_at': '2022-08-13 14:00:00',
   'result_info': 'Game ended in draw.',
   'leg': '1/1',
   'details': None,
   'length': 90,
   'placeholder': False,
   'has_odds': True,
   'has_premium_odds': False,
   'starting_at_timestamp': 1660399200,
   'formations': [{'id': 336854,
     'fixture_id': 18535054,
     'participant_id': 20,
     'formation': '4-3-3',
     'location': 'away'},
    {'id': 336853,
     'fixture_id': 18535054,
     'participant_id': 78,
     'formation': '3-4-2-1',
     'location': 'home'}],
   'weatherreport': {'id': 104269,
    'fixture_id': 18535054,
    'venue_id': 480,
    'temperature': {'day': 29.75,
     'morning': 18.9,
     'evening': 27.83,
     'night': 20.56},
    'feels_lik